In [37]:
import pandas as pd
from pycaret.classification import *

In [64]:
# DB에 연결
import sys
sys.path.append('/Users/nezmi/Projects/yeznable_projects')

from LAB.Data_collection import functions_for_db
from personal import id_pw_classes

mysql_user = id_pw_classes.mysql()

host = '127.0.0.1'
user = mysql_user.id_yeznable
password = mysql_user.pw_yeznable
db = 'GSEnTec'

In [65]:
# create connecttion and cursor
connection = functions_for_db.get_connection(host,user,password,db)
cursor = connection.cursor()
print('DB connected...')

DB connected...


In [72]:
connection.close()

In [66]:
query_get_column = f"""show columns from MART"""
cursor.execute(query_get_column)
columns = cursor.fetchall()
list_columns = []
for column in columns:
    list_columns.append(column[0])
list_columns

['id',
 '부서',
 '사원구분',
 '고용형태',
 '직군',
 '직종',
 '최종학력',
 'pp_employee_class',
 'pp_employment_type',
 'pp_occupation',
 'pp_contract_type',
 'pp_education',
 'pp_gender',
 'pp_incumbent',
 'korean',
 'pp_religion',
 'pp_turnover',
 'pp_ career_rel',
 'pp_ career_etc',
 'pp_ career_max',
 'pp_ career_hm',
 'pp_turnover_self',
 'pp_turnover_other',
 'pp_TOEIC',
 'pp_TOEIC_SPEAKING',
 'pp_OPIc',
 'pp_Lang_etc',
 'pp_lang_level',
 'pp_relation_mother',
 'pp_relation_mate',
 'pp_relation_father',
 'pp_relation_gd',
 'pp_relation_gs',
 'pp_relation_children',
 'pp_relation_sister',
 'pp_relation_ml',
 'pp_relation_fl',
 'pp_relation_gm',
 'pp_relation_gf',
 'pp_relation_brother',
 'pp_relation_family',
 'pp_appointment',
 'pp_appointment_a',
 'pp_appointment_b',
 'pp_appointment_c',
 'pp_appointment_d',
 'pp_appointment_e',
 'pp_appointment_f',
 'pp_department_domain',
 'pp_department_job',
 'pp_address',
 'pp_address_curr',
 'pp_ certificate',
 'pp_ certificate_rel',
 'pp_ certificate_etc',
 'pp_age_join',
 'pp_club',
 'pp_club_a',
 'pp_club_b',
 'pp_club_c',
 'pp_club_d',
 'pp_club_e',
 'pp_club_f',
 'pp_club_g',
 'pp_club_h',
 'pp_club_i',
 'pp_club_j',
 'pp_club_k',
 'pp_longevity',
 'pp_longevity_5y',
 'pp_longevity_10y']

In [5]:
# numeric_features = list_columns[15:22]+list_columns[39:47]+list_columns[51:56]
# categorical_features = list(set(list_columns) - set(numeric_features))
# numeric_features
# # categorical_features

['pp_turnover',
 'pp_ career_rel',
 'pp_ career_etc',
 'pp_ career_max',
 'pp_ career_hm',
 'pp_turnover_self',
 'pp_turnover_other',
 'pp_relation_family',
 'pp_appointment',
 'pp_appointment_a',
 'pp_appointment_b',
 'pp_appointment_c',
 'pp_appointment_d',
 'pp_appointment_e',
 'pp_appointment_f',
 'pp_ certificate',
 'pp_ certificate_rel',
 'pp_ certificate_etc',
 'pp_age_join',
 'pp_club']

In [62]:
numeric_features = list_columns[16:23]+list_columns[40:48]+list_columns[52:57]
categorical_features = list(set(list_columns) - set(numeric_features))
# numeric_features
categorical_features

['사원구분',
 'pp_club_e',
 'pp_department_job',
 'pp_relation_ml',
 'pp_club_j',
 'pp_club_b',
 'pp_address_curr',
 'pp_relation_mate',
 'pp_religion',
 'pp_relation_gd',
 'pp_club_i',
 '최종학력',
 'pp_club_h',
 'pp_relation_sister',
 'pp_club_k',
 'pp_TOEIC',
 'pp_relation_mother',
 'pp_lang_level',
 'pp_gender',
 'pp_occupation',
 'pp_club_f',
 '부서',
 '직군',
 'pp_longevity_10y',
 'pp_relation_gm',
 'pp_relation_gf',
 'pp_incumbent',
 'pp_address',
 'pp_education',
 'pp_longevity',
 'pp_relation_gs',
 'pp_club_g',
 'pp_club_c',
 'pp_relation_fl',
 'pp_employee_class',
 'pp_department_domain',
 'pp_relation_father',
 'pp_club_a',
 'pp_Lang_etc',
 'id',
 'pp_relation_children',
 'pp_club_d',
 'pp_employment_type',
 'korean',
 '고용형태',
 'pp_OPIc',
 'pp_relation_brother',
 '직종',
 'pp_contract_type',
 'pp_TOEIC_SPEAKING',
 'pp_longevity_5y']

In [67]:
# str_columns = ''
# for column in list_columns[17:]:
#     str_columns = str_columns + column + ','
    
query_get_data = f"""SELECT * FROM MART WHERE pp_longevity_10y = 1 OR pp_incumbent = 0"""
# print(query_get_data)

In [68]:
cursor.execute(query_get_data)
data = cursor.fetchall()
df = pd.DataFrame(data)
df

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
0,1,화공영업,연봉(직원),상용직,임원,정규직,대학원졸,salary_plain,regular,executives,...,0,0,0,1,0,0,0,370,1,1
1,2,후공정,연봉(현장),상용직,생산직(현장),정규직,대졸,salary_field,regular,production,...,0,0,0,1,0,0,0,354,1,1
2,3,선행,연봉(현장),상용직,기술직,정규직,대졸,salary_field,regular,technical,...,0,0,0,1,0,0,0,264,1,1
3,4,자재관리,연봉(직원),상용직,사무직,정규직,대졸,salary_plain,regular,white_collar,...,0,1,0,0,0,1,0,258,1,1
4,5,설비관리,시급(현장),상용직,사무직,정규직,대졸,hourly_field,regular,white_collar,...,0,0,0,0,0,0,0,211,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,1489,일용직,일급,일용직,일용직,일용직,empty,daily,daily,daily,...,0,0,0,1,0,0,0,0,0,0
1234,1490,일용직,일급,일용직,일용직,일용직,empty,daily,daily,daily,...,0,0,0,1,0,0,0,0,0,0
1235,1491,일용직,일급,일용직,일용직,일용직,empty,daily,daily,daily,...,0,0,0,1,0,0,0,0,0,0
1236,1492,일용직,일급,일용직,일용직,일용직,empty,daily,daily,daily,...,0,0,0,1,0,0,0,0,0,0


In [69]:
df.columns = list_columns
# df.to_csv('data2.csv', encoding='euc-kr')
df

,id,부서,사원구분,고용형태,직군,직종,최종학력,pp_employee_class,pp_employment_type,pp_occupation,...,pp_club_e,pp_club_f,pp_club_g,pp_club_h,pp_club_i,pp_club_j,pp_club_k,pp_longevity,pp_longevity_5y,pp_longevity_10y
0,1,화공영업,연봉(직원),상용직,임원,정규직,대학원졸,salary_plain,regular,executives,...,0,0,0,1,0,0,0,370,1,1
1,2,후공정,연봉(현장),상용직,생산직(현장),정규직,대졸,salary_field,regular,production,...,0,0,0,1,0,0,0,354,1,1
2,3,선행,연봉(현장),상용직,기술직,정규직,대졸,salary_field,regular,technical,...,0,0,0,1,0,0,0,264,1,1
3,4,자재관리,연봉(직원),상용직,사무직,정규직,대졸,salary_plain,regular,white_collar,...,0,1,0,0,0,1,0,258,1,1
4,5,설비관리,시급(현장),상용직,사무직,정규직,대졸,hourly_field,regular,white_collar,...,0,0,0,0,0,0,0,211,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,1489,일용직,일급,일용직,일용직,일용직,empty,daily,daily,daily,...,0,0,0,1,0,0,0,0,0,0
1234,1490,일용직,일급,일용직,일용직,일용직,empty,daily,daily,daily,...,0,0,0,1,0,0,0,0,0,0
1235,1491,일용직,일급,일용직,일용직,일용직,empty,daily,daily,daily,...,0,0,0,1,0,0,0,0,0,0
1236,1492,일용직,일급,일용직,일용직,일용직,empty,daily,daily,daily,...,0,0,0,1,0,0,0,0,0,0


In [70]:
df.drop(['id','부서','사원구분','고용형태', '직군', '직종', '최종학력', 'pp_incumbent', 'pp_longevity', 'pp_longevity_5y'],
       axis = 1, inplace = True)
display(df)
df['pp_longevity_10y'].value_counts()

,pp_employee_class,pp_employment_type,pp_occupation,pp_contract_type,pp_education,pp_gender,korean,pp_religion,pp_turnover,pp_ career_rel,...,pp_club_c,pp_club_d,pp_club_e,pp_club_f,pp_club_g,pp_club_h,pp_club_i,pp_club_j,pp_club_k,pp_longevity_10y
0,salary_plain,regular,executives,regular,graduate,M,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
1,salary_field,regular,production,regular,bachelor,M,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,salary_field,regular,technical,regular,bachelor,M,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,salary_plain,regular,white_collar,regular,bachelor,M,1,1,0,0,...,0,0,0,1,0,0,0,1,0,1
4,hourly_field,regular,white_collar,regular,bachelor,M,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,daily,daily,daily,daily,empty,M,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1234,daily,daily,daily,daily,empty,M,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1235,daily,daily,daily,daily,empty,M,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1236,daily,daily,daily,daily,empty,M,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


0    1143
1      95
Name: pp_longevity_10y, dtype: int64

In [71]:
# df.to_csv('data_alpha.csv', encoding='utf-8')

In [ ]:
# exp_fix_imbal = setup(data = df, target = 'pp_longevity_10y', session_id=123, fix_imbalance = True, numeric_features = numeric_features)
# exp_clf101 = setup(data = df, target = 'pp_longevity_10y', session_id=123)

In [ ]:
# model = create_model('xgboost')

In [ ]:
# print(model)

In [ ]:
# import matplotlib
# from matplotlib import font_manager, rc

# rc('font', family='AppleGothic')
    
# matplotlib.rcParams['axes.unicode_minus'] = False

In [ ]:
# evaluate_model(model)

In [73]:
exp_without_fix_imbal = setup(data = df, target = 'pp_longevity_10y', session_id=123, numeric_features = numeric_features)

Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(1238, 61)"
4,Missing Values,False
5,Numeric Features,20
6,Categorical Features,40
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [74]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Light Gradient Boosting Machine,0.9780,0.9627,0.7690,0.9133,0.8287,0.8174,0.8242,0.1244
1,Extreme Gradient Boosting,0.9757,0.9648,0.7405,0.9365,0.8078,0.7958,0.8117,1.1627
2,CatBoost Classifier,0.9734,0.9797,0.6619,0.9750,0.7734,0.7606,0.7841,7.1253
3,Logistic Regression,0.9723,0.9629,0.7262,0.8798,0.7894,0.7751,0.7825,0.1237
4,Gradient Boosting Classifier,0.9723,0.9647,0.6619,0.9467,0.7672,0.7535,0.7734,0.4603
5,Ada Boost Classifier,0.9676,0.9461,0.7095,0.8395,0.7609,0.7441,0.7514,0.2826
6,Extra Trees Classifier,0.9642,0.9403,0.5738,0.9350,0.7011,0.6838,0.7115,0.3949
7,Random Forest Classifier,0.9630,0.9388,0.5595,0.9350,0.6743,0.6580,0.6923,0.1453
8,Ridge Classifier,0.9619,0.0000,0.6643,0.7981,0.7166,0.6966,0.7045,0.0258
9,Linear Discriminant Analysis,0.9619,0.9269,0.7119,0.7695,0.7362,0.7158,0.7183,0.0636


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [75]:
model_xg = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9885,1.0000,1.0000,0.8750,0.9333,0.9271,0.9295
1,0.9770,0.9982,0.7143,1.0000,0.8333,0.8214,0.8348
2,0.9655,0.9607,0.5714,1.0000,0.7273,0.7103,0.7421
3,0.9770,0.9036,0.7143,1.0000,0.8333,0.8214,0.8348
4,0.9770,0.9929,0.8571,0.8571,0.8571,0.8446,0.8446
5,0.9770,0.9071,0.7143,1.0000,0.8333,0.8214,0.8348
6,0.9535,0.9146,0.3333,1.0000,0.5000,0.4819,0.5634
7,0.9767,0.9771,0.8333,0.8333,0.8333,0.8208,0.8208
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
9,0.9651,0.9938,0.6667,0.8000,0.7273,0.7088,0.7122


In [78]:
evaluate_model(model_xg)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [76]:
model_lgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9885,1.0000,0.8571,1.0000,0.9231,0.9169,0.9201
2,0.9770,0.9732,0.7143,1.0000,0.8333,0.8214,0.8348
3,0.9655,0.8911,0.7143,0.8333,0.7692,0.7507,0.7533
4,0.9885,1.0000,0.8571,1.0000,0.9231,0.9169,0.9201
5,0.9770,0.9607,0.7143,1.0000,0.8333,0.8214,0.8348
6,0.9419,0.8479,0.3333,0.6667,0.4444,0.4173,0.4455
7,0.9651,0.9625,0.6667,0.8000,0.7273,0.7088,0.7122
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
9,0.9767,0.9917,0.8333,0.8333,0.8333,0.8208,0.8208


In [77]:
evaluate_model(model_lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

# Real Flow

In [179]:
numeric_features = ['pp_turnover',
 'pp_ career_rel',
 'pp_ career_etc',
 'pp_ career_max',
 'pp_ career_hm',
 'pp_turnover_self',
 'pp_turnover_other',
 'pp_relation_family',
 'pp_appointment_a',
 'pp_appointment_b',
 'pp_appointment_c',
 'pp_appointment_d',
 'pp_appointment_e',
 'pp_appointment_f',
 'pp_ certificate',
 'pp_ certificate_rel',
 'pp_ certificate_etc',
 'pp_age_join',
 'pp_club']

# float들을 int로 바꿔줘야 된다.
dict_round = {}
for column in numeric_features:
    dict_round[column] = 0

### raw 데이터를 불러오고 test에 활용할 10%를 떼어놓는다.

In [200]:
df_alpha = pd.read_csv(r'./data_alpha.csv', encoding='utf-8')
df_alpha.drop(['Unnamed: 0'], axis='columns', inplace=True)    
display(df_alpha)
print(df_alpha['pp_longevity_10y'].value_counts())

test_set = df_alpha.sample(frac=0.1, random_state=123)
print(test_set['pp_longevity_10y'].value_counts())

train_set = df_alpha.drop(test_set.index)
print(train_set['pp_longevity_10y'].value_counts())

,pp_employee_class,pp_employment_type,pp_occupation,pp_contract_type,pp_education,pp_gender,korean,pp_religion,pp_turnover,pp_ career_rel,pp_ career_etc,pp_ career_max,pp_ career_hm,pp_turnover_self,pp_turnover_other,pp_TOEIC,pp_TOEIC_SPEAKING,pp_OPIc,pp_Lang_etc,pp_lang_level,pp_relation_mother,pp_relation_mate,pp_relation_father,pp_relation_gd,pp_relation_gs,pp_relation_children,pp_relation_sister,pp_relation_ml,pp_relation_fl,pp_relation_gm,pp_relation_gf,pp_relation_brother,pp_relation_family,pp_appointment_a,pp_appointment_b,pp_appointment_c,pp_appointment_d,pp_appointment_e,pp_appointment_f,pp_department_domain,pp_department_job,pp_address,pp_address_curr,pp_ certificate,pp_ certificate_rel,pp_ certificate_etc,pp_age_join,pp_club,pp_club_a,pp_club_b,pp_club_c,pp_club_d,pp_club_e,pp_club_f,pp_club_g,pp_club_h,pp_club_i,pp_club_j,pp_club_k,pp_longevity_10y
0,salary_plain,regular,executives,regular,graduate,M,1,0,0,0,0,0,0.0,0,0,0,0,0,0,N,0,1,0,0,0,3,0,1,0,0,0,0,5,0,0,0,0,0,0.000000,ChemicalEngeneering,Sales,ulsan,ulsan,1,1,0,25,2,0,0,0,1,0,0,0,1,0,0,0,1
1,salary_field,regular,production,regular,bachelor,M,1,0,0,0,0,0,0.0,0,0,0,0,0,0,N,0,1,0,0,0,2,0,0,0,0,0,0,3,0,0,0,4,0,0.011299,PostRrocess,empty,ulsan,ulsan,0,0,0,29,0,0,0,0,0,0,0,0,1,0,0,0,1
2,salary_field,regular,technical,regular,bachelor,M,1,0,0,0,0,0,0.0,0,0,0,0,0,0,N,1,1,1,0,0,2,0,0,0,0,0,0,5,0,0,0,0,0,0.000000,empty,Preceding,ulsan,ulsan,0,0,0,24,0,0,0,0,0,0,0,0,1,0,0,0,1
3,salary_plain,regular,white_collar,regular,bachelor,M,1,1,0,0,0,0,0.0,0,0,0,0,0,0,N,0,1,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0.000000,Meterials,Management,ulsan,ulsan,0,0,0,28,2,0,0,0,0,0,1,0,0,0,1,0,1
4,hourly_field,regular,white_collar,regular,bachelor,M,1,0,0,0,0,0,0.0,0,0,0,0,0,0,N,0,1,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0.000000,Facilities,Management,ulsan,ulsan,0,0,0,37,1,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,daily,daily,daily,daily,empty,M,1,0,0,0,0,0,0.0,0,0,0,0,0,0,N,1,1,0,0,0,2,0,0,0,0,0,0,4,0,0,0,0,0,0.000000,empty,Daily,busan,busan,0,0,0,31,0,0,0,0,0,0,0,0,1,0,0,0,0
1234,daily,daily,daily,daily,empty,M,1,0,0,0,0,0,0.0,0,0,0,0,0,0,N,1,1,0,0,0,2,0,0,0,0,0,0,4,0,0,0,0,0,0.000000,empty,Daily,busan,busan,0,0,0,31,0,0,0,0,0,0,0,0,1,0,0,0,0
1235,daily,daily,daily,daily,empty,M,1,0,0,0,0,0,0.0,0,0,0,0,0,0,N,1,0,1,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0.000000,empty,Daily,busan,busan,0,0,0,30,0,0,0,0,0,0,0,0,1,0,0,0,0
1236,daily,daily,daily,daily,empty,M,1,0,0,0,0,0,0.0,0,0,0,0,0,0,N,1,0,1,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0.000000,empty,Daily,ulsan,ulsan,0,0,0,24,0,0,0,0,0,0,0,0,1,0,0,0,0


0    1143
1      95
Name: pp_longevity_10y, dtype: int64
0    113
1     11
Name: pp_longevity_10y, dtype: int64
0    1030
1      84
Name: pp_longevity_10y, dtype: int64


In [201]:
test_set_label = test_set.copy()
print(test_set_label.shape)
test_set.drop(['pp_longevity_10y'], axis='columns', inplace=True)
print(test_set.shape)
print(test_set_label.shape)

(124, 60)
(124, 59)
(124, 60)


### SMOTE와 CTGAN의 성능을 비교한다.

#### CTGAN을 활용한 balanced data 만들기

In [202]:
df_syn = pd.read_csv(r'./ctgan_data_eval/data_syn.csv', encoding='utf-8')
df_syn.drop(['Unnamed: 0', 'pp_appointment'], axis='columns', inplace=True)    
df_syn = df_syn.round(dict_round)

train_1 = train_set.loc[train_set['pp_longevity_10y'] == 1]
syn_1 = df_syn.loc[df_syn['pp_longevity_10y'] == 1]

syn_sample = syn_1.sample(n=train_set['pp_longevity_10y'].value_counts()[0]-train_set['pp_longevity_10y'].value_counts()[1],
                                random_state=123)
train_bal_ctgan = pd.concat([train_set, syn_sample])   
print(train_bal_ctgan['pp_longevity_10y'].value_counts())
print(train_set['pp_longevity_10y'].value_counts())

1    1030
0    1030
Name: pp_longevity_10y, dtype: int64
0    1030
1      84
Name: pp_longevity_10y, dtype: int64


#### SMOTE를 활용한 balanced data 만들기

In [162]:
from imblearn.over_sampling import SMOTENC

In [213]:
X_imb = train_set[list(set(train_set.columns) - {'pp_longevity_10y'})]
y_imb = train_set[['pp_longevity_10y']]

In [215]:
categorical_features = list(set(X_imb.columns) - set(numeric_features))

categorical_features_index = []
for cat in categorical_features:
    categorical_features_index.append(list(X_imb.columns).index(cat))
print(categorical_features,'\n',categorical_features_index)

['pp_club_e', 'pp_department_job', 'pp_relation_ml', 'pp_club_j', 'pp_club_b', 'pp_address_curr', 'pp_relation_mate', 'pp_religion', 'pp_relation_gd', 'pp_club_i', 'pp_club_h', 'pp_relation_sister', 'pp_club_k', 'pp_TOEIC', 'pp_relation_mother', 'pp_lang_level', 'pp_gender', 'pp_occupation', 'pp_club_f', 'pp_relation_gm', 'pp_relation_gf', 'pp_address', 'pp_education', 'pp_relation_gs', 'pp_club_g', 'pp_club_c', 'pp_relation_fl', 'pp_employee_class', 'pp_department_domain', 'pp_relation_father', 'pp_club_a', 'pp_relation_children', 'pp_club_d', 'pp_employment_type', 'korean', 'pp_OPIc', 'pp_relation_brother', 'pp_contract_type', 'pp_TOEIC_SPEAKING', 'pp_Lang_etc'] 
 [0, 5, 4, 6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 27, 29, 30, 31, 34, 36, 37, 38, 39, 41, 42, 43, 46, 52, 48, 49, 51, 54, 55, 57, 58]


In [214]:
# display(X_imb['pp_longevity_10y'])
# len(X_imb.columns)

print(len(list(set(train_set.columns) - set(['pp_longevity_10y']))), len(train_set.columns))

59 60


In [216]:
smote_nc = SMOTENC(categorical_features=categorical_features_index, random_state=123)
X_resampled, y_resampled = smote_nc.fit_resample(X_imb, y_imb)
print(y_resampled['pp_longevity_10y'].value_counts())

1    1030
0    1030
Name: pp_longevity_10y, dtype: int64


In [217]:
train_bal_smote = pd.concat([X_resampled, y_resampled], axis = 1)
display(train_bal_smote)
display(train_bal_ctgan)

,pp_club_e,pp_ career_etc,pp_appointment_d,pp_relation_family,pp_relation_ml,pp_department_job,pp_club_j,pp_club_b,pp_turnover,pp_address_curr,pp_relation_mate,pp_religion,pp_relation_gd,pp_appointment_c,pp_club_i,pp_club_h,pp_relation_sister,pp_club_k,pp_TOEIC,pp_relation_mother,pp_ career_max,pp_lang_level,pp_gender,pp_occupation,pp_club_f,pp_ career_rel,pp_appointment_f,pp_relation_gm,pp_appointment_a,pp_relation_gf,pp_address,pp_education,pp_turnover_self,pp_age_join,pp_relation_gs,pp_ certificate_rel,pp_club_g,pp_club_c,pp_relation_fl,pp_employee_class,pp_ certificate,pp_department_domain,pp_relation_father,pp_club_a,pp_club,pp_appointment_b,pp_relation_children,pp_ certificate_etc,pp_employment_type,korean,pp_appointment_e,pp_OPIc,pp_club_d,pp_ career_hm,pp_relation_brother,pp_contract_type,pp_turnover_other,pp_TOEIC_SPEAKING,pp_Lang_etc,pp_longevity_10y
0,0,0,0,5,1,Sales,0,0,0,ulsan,1,0,0,0,0,1,0,0,0,0,0,N,M,executives,0,0,0.000000,0,0,0,ulsan,graduate,0,25,0,1,0,0,0,salary_plain,1,ChemicalEngeneering,0,0,2,0,3,0,regular,1,0,0,1,0.000000,0,regular,0,0,0,1
1,0,0,4,3,0,empty,0,0,0,ulsan,1,0,0,0,0,1,0,0,0,0,0,N,M,production,0,0,0.011299,0,0,0,ulsan,bachelor,0,29,0,0,0,0,0,salary_field,0,PostRrocess,0,0,0,0,2,0,regular,1,0,0,0,0.000000,0,regular,0,0,0,1
2,0,0,0,5,0,Preceding,0,0,0,ulsan,1,0,0,0,0,1,0,0,0,1,0,N,M,technical,0,0,0.000000,0,0,0,ulsan,bachelor,0,24,0,0,0,0,0,salary_field,0,empty,1,0,0,0,2,0,regular,1,0,0,0,0.000000,0,regular,0,0,0,1
3,0,0,0,3,0,Management,1,0,0,ulsan,1,1,0,0,0,0,0,0,0,0,0,N,M,white_collar,1,0,0.000000,0,0,0,ulsan,bachelor,0,28,0,0,0,0,0,salary_plain,0,Meterials,0,0,2,0,2,0,regular,1,0,0,0,0.000000,0,regular,0,0,0,1
4,0,0,0,3,0,Management,0,0,0,ulsan,1,0,0,0,0,0,0,0,0,0,0,N,M,white_collar,0,0,0.000000,0,0,0,ulsan,bachelor,0,37,0,0,0,0,0,hourly_field,0,Facilities,0,0,1,0,2,0,regular,1,0,0,1,0.000000,0,regular,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,0,0,1,4,0,Design,0,0,0,ulsan,1,0,0,0,0,1,0,0,1,1,0,L,M,management,0,0,0.014330,0,0,0,ulsan,bachelor,0,27,0,3,0,0,0,salary_plain,3,empty,1,0,0,0,1,0,regular,1,0,0,0,0.000000,1,regular,0,0,0,1
2056,0,1,1,5,0,Production,0,0,1,ulsan,1,0,0,0,0,0,0,0,0,1,14,N,M,management,0,0,0.007690,0,0,0,ulsan,bachelor,1,29,0,0,0,0,0,salary_plain,0,empty,1,0,1,0,2,0,regular,1,0,0,0,14.938818,1,regular,0,0,0,1
2057,0,3,1,3,0,Production,0,0,3,ulsan,1,0,0,0,0,1,0,0,0,1,26,N,M,management,0,0,0.012387,0,0,0,ulsan,bachelor,3,30,0,1,0,0,0,salary_plain,1,Plant,1,0,1,0,2,0,regular,1,0,0,0,17.445343,0,regular,0,0,0,1
2058,0,1,0,3,0,Development,0,0,1,ulsan,1,0,0,0,0,0,0,0,0,0,33,N,M,technical,0,0,0.000728,0,0,0,ulsan,bachelor,1,31,0,0,0,0,0,salary_plain,0,empty,0,0,0,0,2,0,regular,1,0,0,0,30.271539,0,regular,0,0,0,1


,pp_employee_class,pp_employment_type,pp_occupation,pp_contract_type,pp_education,pp_gender,korean,pp_religion,pp_turnover,pp_ career_rel,pp_ career_etc,pp_ career_max,pp_ career_hm,pp_turnover_self,pp_turnover_other,pp_TOEIC,pp_TOEIC_SPEAKING,pp_OPIc,pp_Lang_etc,pp_lang_level,pp_relation_mother,pp_relation_mate,pp_relation_father,pp_relation_gd,pp_relation_gs,pp_relation_children,pp_relation_sister,pp_relation_ml,pp_relation_fl,pp_relation_gm,pp_relation_gf,pp_relation_brother,pp_relation_family,pp_appointment_a,pp_appointment_b,pp_appointment_c,pp_appointment_d,pp_appointment_e,pp_appointment_f,pp_department_domain,pp_department_job,pp_address,pp_address_curr,pp_ certificate,pp_ certificate_rel,pp_ certificate_etc,pp_age_join,pp_club,pp_club_a,pp_club_b,pp_club_c,pp_club_d,pp_club_e,pp_club_f,pp_club_g,pp_club_h,pp_club_i,pp_club_j,pp_club_k,pp_longevity_10y
0,salary_plain,regular,executives,regular,graduate,M,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,N,0,1,0,0,0,3,0,1,0,0,0,0,5.0,0.0,0.0,0.0,0.0,0.0,0.000000,ChemicalEngeneering,Sales,ulsan,ulsan,1.0,1.0,0.0,25.0,2.0,0,0,0,1,0,0,0,1,0,0,0,1
1,salary_field,regular,production,regular,bachelor,M,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,N,0,1,0,0,0,2,0,0,0,0,0,0,3.0,0.0,0.0,0.0,4.0,0.0,0.011299,PostRrocess,empty,ulsan,ulsan,0.0,0.0,0.0,29.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1
2,salary_field,regular,technical,regular,bachelor,M,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,N,1,1,1,0,0,2,0,0,0,0,0,0,5.0,0.0,0.0,0.0,0.0,0.0,0.000000,empty,Preceding,ulsan,ulsan,0.0,0.0,0.0,24.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1
3,salary_plain,regular,white_collar,regular,bachelor,M,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,N,0,1,0,0,0,2,0,0,0,0,0,0,3.0,0.0,0.0,0.0,0.0,0.0,0.000000,Meterials,Management,ulsan,ulsan,0.0,0.0,0.0,28.0,2.0,0,0,0,0,0,1,0,0,0,1,0,1
4,hourly_field,regular,white_collar,regular,bachelor,M,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,N,0,1,0,0,0,2,0,0,0,0,0,0,3.0,0.0,0.0,0.0,0.0,0.0,0.000000,Facilities,Management,ulsan,ulsan,0.0,0.0,0.0,37.0,1.0,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10016,hourly_field,regular,management,regular,bachelor,M,1,0,3.0,-0.0,3.0,3.0,15.0,3.0,-0.0,0,0,0,0,N,1,0,1,0,0,0,0,0,0,0,0,2,4.0,0.0,-0.0,-0.0,5.0,0.0,-0.000000,HSE,QC,ulsan,ulsan,1.0,1.0,0.0,26.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1
14760,salary_plain,regular,management,regular,college,M,1,0,-0.0,-0.0,0.0,2.0,1.0,-0.0,0.0,0,0,0,0,L,0,1,1,0,0,1,0,0,0,0,0,0,5.0,-0.0,-0.0,0.0,4.0,2.0,0.000000,empty,Design,ulsan,ulsan,1.0,2.0,0.0,32.0,0.0,0,0,0,0,0,0,0,0,0,1,0,1
11777,salary_plain,regular,research,regular,bachelor,M,1,0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0,0,0,0,N,1,0,1,0,0,0,0,0,0,0,0,2,5.0,-0.0,-0.0,-0.0,1.0,0.0,0.000000,empty,Preceding,ulsan,ulsan,1.0,3.0,-0.0,23.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1
2363,salary_plain,regular,executives,regular,bachelor,M,1,0,3.0,2.0,3.0,54.0,73.0,3.0,-0.0,0,0,0,0,N,0,1,0,0,0,2,0,0,0,0,0,0,3.0,0.0,0.0,0.0,1.0,-0.0,0.000000,empty,PM,gyeonggi,seoul,0.0,0.0,0.0,35.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1


In [180]:
setup_CTGAN = setup(data = train_bal_ctgan, target = 'pp_longevity_10y', session_id=123, numeric_features = numeric_features)

Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(2060, 60)"
4,Missing Values,False
5,Numeric Features,19
6,Categorical Features,40
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [181]:
model_lgbm_ctgan = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9724,0.9933,0.9726,0.9726,0.9726,0.9448,0.9448
1,0.9514,0.9909,0.9444,0.9577,0.9510,0.9028,0.9029
2,0.9722,0.9948,0.9722,0.9722,0.9722,0.9444,0.9444
3,0.9583,0.9892,0.9306,0.9853,0.9571,0.9167,0.9181
4,0.9722,0.9931,0.9722,0.9722,0.9722,0.9444,0.9444
5,0.9583,0.9996,0.9167,1.0000,0.9565,0.9167,0.9199
6,0.9514,0.9880,0.9583,0.9452,0.9517,0.9028,0.9029
7,0.9444,0.9875,0.9306,0.9571,0.9437,0.8889,0.8892
8,0.9653,0.9927,0.9722,0.9589,0.9655,0.9306,0.9306
9,0.9861,0.9994,0.9722,1.0000,0.9859,0.9722,0.9726


In [90]:
setup_SMOTE = setup(data = train_set, target = 'pp_longevity_10y', session_id=123, fix_imbalance = True, numeric_features = numeric_features)

Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(1114, 61)"
4,Missing Values,False
5,Numeric Features,20
6,Categorical Features,40
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [91]:
model_lgbm_smote = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9615,0.9398,0.6667,0.8000,0.7273,0.7068,0.7102
1,0.9872,0.9560,0.8333,1.0000,0.9091,0.9023,0.9066
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.9359,0.9769,0.3333,0.6667,0.4444,0.4144,0.4426
4,0.9872,0.9977,0.8333,1.0000,0.9091,0.9023,0.9066
5,0.9615,0.9769,0.6667,0.8000,0.7273,0.7068,0.7102
6,0.9744,0.9931,0.6667,1.0000,0.8000,0.7869,0.8054
7,0.9744,0.9884,0.8333,0.8333,0.8333,0.8194,0.8194
8,0.9872,1.0000,0.8333,1.0000,0.9091,0.9023,0.9066
9,0.9740,0.9667,0.6000,1.0000,0.7500,0.7372,0.7641


In [218]:
setup_SMOTE = setup(data = train_bal_smote, target = 'pp_longevity_10y', session_id=123, numeric_features = numeric_features)

Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(2060, 60)"
4,Missing Values,False
5,Numeric Features,19
6,Categorical Features,40
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [219]:
model_lgbm_smote = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9862,0.9983,0.9726,1.0000,0.9861,0.9724,0.9728
1,0.9931,1.0000,1.0000,0.9863,0.9931,0.9861,0.9862
2,0.9861,0.9998,1.0000,0.9730,0.9863,0.9722,0.9726
3,0.9653,0.9979,0.9722,0.9589,0.9655,0.9306,0.9306
4,0.9653,0.9977,0.9861,0.9467,0.9660,0.9306,0.9314
5,0.9931,1.0000,0.9861,1.0000,0.9930,0.9861,0.9862
6,0.9722,0.9973,0.9583,0.9857,0.9718,0.9444,0.9448
7,0.9931,0.9994,0.9861,1.0000,0.9930,0.9861,0.9862
8,0.9931,0.9961,0.9861,1.0000,0.9930,0.9861,0.9862
9,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [ ]:
# finalize a model
lr_final = finalize_model(lr)
# importing unseen data 
import pandas as pd
data_unseen = pd.read_csv('data_unseen.csv') 
# generate predictions on unseen data
predictions = predict_model(lr_final, data = data_unseen)

In [92]:
lgbm_ctgn_final = finalize_model(model_lgbm_ctgan)

In [103]:
predictions = predict_model(lgbm_ctgn_final, data = test_set)

In [128]:
display(test_set_label)
display(predictions)

,pp_employee_class,pp_employment_type,pp_occupation,pp_contract_type,pp_education,pp_gender,korean,pp_religion,pp_turnover,pp_ career_rel,...,pp_club_c,pp_club_d,pp_club_e,pp_club_f,pp_club_g,pp_club_h,pp_club_i,pp_club_j,pp_club_k,pp_longevity_10y
163,salary_plain,regular,technical,regular,bachelor,M,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
394,salary_plain,regular,technical,regular,bachelor,M,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
633,salary_plain,regular,white_collar,regular,graduate,M,1,1,1,0,...,0,0,0,0,0,1,0,0,0,0
373,salary_plain,regular,research,regular,bachelor,F,1,0,2,0,...,0,0,0,0,0,0,0,0,0,0
597,salary_plain,regular,white_collar,regular,bachelor,M,1,1,3,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,hourly_field,regular,production,temporary,high_school,M,1,0,4,0,...,0,0,0,0,0,1,0,0,0,0
644,salary_plain,regular,white_collar,regular,bachelor,M,1,0,2,0,...,0,0,0,0,0,1,0,0,0,0
714,salary_plain,regular,research,regular,bachelor,F,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
252,salary_plain,regular,management,regular,bachelor,M,1,0,2,0,...,0,0,0,0,0,1,0,0,0,1


,pp_employee_class,pp_employment_type,pp_occupation,pp_contract_type,pp_education,pp_gender,korean,pp_religion,pp_turnover,pp_ career_rel,...,pp_club_d,pp_club_e,pp_club_f,pp_club_g,pp_club_h,pp_club_i,pp_club_j,pp_club_k,Label,Score
0,salary_plain,regular,technical,regular,bachelor,M,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0000
1,salary_plain,regular,technical,regular,bachelor,M,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0.0008
2,salary_plain,regular,white_collar,regular,graduate,M,1,1,1,0,...,0,0,0,0,1,0,0,0,0,0.0000
3,salary_plain,regular,research,regular,bachelor,F,1,0,2,0,...,0,0,0,0,0,0,0,0,0,0.0015
4,salary_plain,regular,white_collar,regular,bachelor,M,1,1,3,0,...,0,0,0,0,1,0,0,0,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,hourly_field,regular,production,temporary,high_school,M,1,0,4,0,...,0,0,0,0,1,0,0,0,0,0.0000
120,salary_plain,regular,white_collar,regular,bachelor,M,1,0,2,0,...,0,0,0,0,1,0,0,0,0,0.0000
121,salary_plain,regular,research,regular,bachelor,F,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0000
122,salary_plain,regular,management,regular,bachelor,M,1,0,2,0,...,0,0,0,0,1,0,0,0,1,0.8400


In [133]:
for i, row in enumerate(test_set_label['pp_longevity_10y']):
    print(i, row)
    break

0 0
